<a href="https://colab.research.google.com/github/CHRISTIANDAGATTI/AEyCD---Grupo-9/blob/main/Entregable_parte_1_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2022*

----

# Trabajo práctico entregable - parte 1

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

In [1]:
!pip install sqlalchemy

In [2]:
!pip install decouple

  Created wheel for decouple: filename=decouple-0.0.7-py3-none-any.whl size=4483 sha256=87ae997193b3d585ba4c609cf38ee583372cbd086bef62701b9c1dc6f63f399c
  Stored in directory: /root/.cache/pip/wheels/c0/f8/ba/513f9e16fec0df6b2dfd693ae2c057773e912c478b6019f24f
Successfully built decouple


In [3]:
!pip install missingno

In [4]:
!pip install plotly

In [5]:
!pip install config

In [6]:
import matplotlib.pyplot as plt
import numpy
import pandas

import seaborn
seaborn.set_context('talk')


import config
from sqlalchemy import create_engine, text

In [7]:
import plotly
plotly.__version__
# Make sure it's 4.14.3

'5.5.0'

In [8]:
melb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
melb_df[:3]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


## Ejercicio 1 SQL: 

1. Crear una base de datos en SQLite utilizando la libreria [SQLalchemy](https://stackoverflow.com/questions/2268050/execute-sql-from-file-in-sqlalchemy).
https://docs.sqlalchemy.org/en/14/core/engines.html#sqlite

2. Ingestar los datos provistos en 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv' en una tabla y el dataset generado en clase con datos de airbnb y sus precios por codigo postal en otra.

3. Implementar consultas en SQL que respondan con la siguiente información:

    - cantidad de registros totales por ciudad.
    - cantidad de registros totales por barrio y ciudad.

4. Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL  para obtener un resultado similar a lo realizado con Pandas en clase.  



**1- Creación de base de datos:**

In [9]:
engine = create_engine('sqlite:///sysarmy.sqlite3', echo=True)

**2- Creación de tablas:**

In [10]:
melb_df.to_sql('Melbo', con=engine, if_exists="replace")

2022-05-18 21:10:01,328 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Melbo")
2022-05-18 21:10:01,330 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-18 21:10:01,333 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Melbo")
2022-05-18 21:10:01,336 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-18 21:10:01,344 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-18 21:10:01,348 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Melbo" (
	"index" BIGINT, 
	"Suburb" TEXT, 
	"Address" TEXT, 
	"Rooms" BIGINT, 
	"Type" TEXT, 
	"Price" FLOAT, 
	"Method" TEXT, 
	"SellerG" TEXT, 
	"Date" TEXT, 
	"Distance" FLOAT, 
	"Postcode" FLOAT, 
	"Bedroom2" FLOAT, 
	"Bathroom" FLOAT, 
	"Car" FLOAT, 
	"Landsize" FLOAT, 
	"BuildingArea" FLOAT, 
	"YearBuilt" FLOAT, 
	"CouncilArea" TEXT, 
	"Lattitude" FLOAT, 
	"Longtitude" FLOAT, 
	"Regionname" TEXT, 
	"Propertycount" FLOAT
)


2022-05-18 21:10:01,350 INFO sqlalchemy.engine.Engine [no key 0.00251s] ()
2022-05-18 21:10:01,363 INFO sqlalchemy.engi

In [11]:
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]
airbnb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
    usecols=interesting_cols)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
airbnb_df.to_sql('Airbnb', con=engine, if_exists="replace")

2022-05-18 21:17:24,733 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Airbnb")
2022-05-18 21:17:24,738 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-18 21:17:24,740 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Airbnb")
2022-05-18 21:17:24,743 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-18 21:17:24,749 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-18 21:17:24,753 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Airbnb" (
	"index" BIGINT, 
	description TEXT, 
	neighborhood_overview TEXT, 
	street TEXT, 
	neighborhood TEXT, 
	city TEXT, 
	suburb TEXT, 
	state TEXT, 
	zipcode TEXT, 
	latitude FLOAT, 
	longitude FLOAT, 
	price BIGINT, 
	weekly_price FLOAT, 
	monthly_price FLOAT
)


2022-05-18 21:17:24,756 INFO sqlalchemy.engine.Engine [no key 0.00263s] ()
2022-05-18 21:17:24,772 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Airbnb_index" ON "Airbnb" ("index")
2022-05-18 21:17:24,774 INFO sqlalchemy.engine.Engine [no key 0.00218s] ()
2022-05-18 21:17:24,785

**A traves de "to_numeric" llevamos todos los registros al mismo tipo de datos**

In [13]:
airbnb_df['zipcode'] = pandas.to_numeric(airbnb_df.zipcode, errors='coerce')

In [14]:
airbnb_df.zipcode.value_counts().head(10)

3000.0    3367
3006.0    1268
3182.0    1135
3141.0     881
3004.0     728
3121.0     628
3008.0     616
3053.0     538
3181.0     460
3056.0     446
Name: zipcode, dtype: int64

**3. Consultas SQL:**

**Cantidad de registros totales por ciudad:**


(Guardamos strings en variables y luego pandas lo convierte en una sentencia sql)

In [15]:
query1 = "SELECT city, COUNT(*) FROM Airbnb GROUP BY city ORDER BY COUNT(city) desc"
query2 = "SELECT city, neighborhood, COUNT(city) FROM Airbnb GROUP BY city, neighborhood  ORDER BY city"

In [16]:
with engine.connect() as con:
      rs = con.execute(query1)
      print(query1)
      for row in rs:
          print(row)

2022-05-18 21:19:25,894 INFO sqlalchemy.engine.Engine SELECT city, COUNT(*) FROM Airbnb GROUP BY city ORDER BY COUNT(city) desc
2022-05-18 21:19:25,897 INFO sqlalchemy.engine.Engine [raw sql] ()
SELECT city, COUNT(*) FROM Airbnb GROUP BY city ORDER BY COUNT(city) desc
('Melbourne', 7368)
('Port Phillip', 2808)
('Yarra', 2049)
('Stonnington', 1621)
('Moreland', 967)
('Yarra Ranges', 771)
('Darebin', 698)
('Boroondara', 664)
('Glen Eira', 631)
('Whitehorse', 614)
('Monash', 571)
('Maribyrnong', 436)
('Wyndham', 426)
('Bayside', 375)
('Moonee Valley', 344)
('Manningham', 313)
('Kingston', 309)
('Hobsons Bay', 239)
('Banyule', 203)
('Frankston', 177)
('Knox', 175)
('Hume', 170)
('Casey', 153)
('Greater Dandenong', 147)
('Whittlesea', 137)
('Cardinia', 123)
('Maroondah', 115)
('Brimbank', 108)
('Melton', 95)
('Nillumbik', 88)


In [17]:

with engine.connect() as con:
      rs = con.execute(query2)
      print(query2)
      for row in rs:
          print(row)

2022-05-18 21:21:20,873 INFO sqlalchemy.engine.Engine SELECT city, neighborhood, COUNT(city) FROM Airbnb GROUP BY city, neighborhood  ORDER BY city
2022-05-18 21:21:20,876 INFO sqlalchemy.engine.Engine [raw sql] ()
SELECT city, neighborhood, COUNT(city) FROM Airbnb GROUP BY city, neighborhood  ORDER BY city
('Banyule', None, 174)
('Banyule', 'Alphington', 16)
('Banyule', 'Preston', 8)
('Banyule', 'Thornbury', 5)
('Bayside', None, 170)
('Bayside', 'Bentleigh', 1)
('Bayside', 'Brighton', 203)
('Bayside', 'Elwood', 1)
('Boroondara', None, 30)
('Boroondara', 'Ashburton', 14)
('Boroondara', 'Balwyn', 70)
('Boroondara', 'Balwyn North', 40)
('Boroondara', 'Camberwell', 82)
('Boroondara', 'Canterbury', 24)
('Boroondara', 'Glen Iris', 32)
('Boroondara', 'Hawthorn', 261)
('Boroondara', 'Kew', 109)
('Boroondara', 'Malvern', 2)
('Brimbank', None, 108)
('Cardinia', None, 123)
('Casey', None, 153)
('Darebin', None, 112)
('Darebin', 'Alphington', 22)
('Darebin', 'Coburg', 4)
('Darebin', 'Northcote', 

**Cantidad de registros totales por barrio y ciudad:**

**4.JOIN de Datasets por SQL:**

Controlamos de que tengamos suficientes registros en común como para agregar información relevante a traves del JOIN por Código Postal.

In [18]:
intersection = numpy.intersect1d(
    airbnb_df.zipcode.values, melb_df.Postcode.values, assume_unique=False)
print("Airbnb unique zipcodes", len(airbnb_df.zipcode.unique()))
print("Sales unique zipcodes", len(melb_df.Postcode.unique()))
print("Common zipcodes", len(intersection))

Airbnb unique zipcodes 248
Sales unique zipcodes 198
Common zipcodes 191


Calculamos el porentaje de el total de registros de Postcode o Zipcode que interaccionan (cruzan) entre los distintos Dataset respecto el total de registros.

In [19]:
print('Records in Sales df with corresponding zipcode form Airbnb df',
      melb_df.Postcode.isin(intersection).sum() / len(melb_df))
print('Records in Airbnb df with corresponding zipcode form Sales df',
      airbnb_df.zipcode.isin(intersection).sum() / len(airbnb_df))

Records in Sales df with corresponding zipcode form Airbnb df 0.9985272459499264
Records in Airbnb df with corresponding zipcode form Sales df 0.9302904564315353


Definimos columnas de interes del Dataset de Airbnb y calculamos ciertas métricas de los registros agrupados por Zipcode.

In [20]:
relevant_cols = ['price', 'weekly_price', 'monthly_price', 'zipcode']
## Option 1
# Pass a dictionary where the keys are the original columns to aggregate and
# the values are the operations (or list of operations).
airbnb_price_by_zipcode = airbnb_df[relevant_cols].groupby('zipcode')\
  .agg({'price': ['mean', 'count'], 'weekly_price': 'mean',
        'monthly_price': 'mean'})\
  .reset_index()
# Flatten the two level columns
airbnb_price_by_zipcode.columns = [
  ' '.join(col).strip()
  for col in airbnb_price_by_zipcode.columns.values]
# Rename columns
airbnb_price_by_zipcode = airbnb_price_by_zipcode.rename(
    columns={'price mean': 'airbnb_price_mean',
             'price count': 'airbnb_record_count',
             'weekly_price mean': 'airbnb_weekly_price_mean',
             'monthly_price mean': 'airbnb_monthly_price_mean'}
)

In [21]:
airbnb_price_by_zipcode.head()

,zipcode,airbnb_price_mean,airbnb_record_count,airbnb_weekly_price_mean,airbnb_monthly_price_mean
0,2010.0,40.000000,1,NaN,NaN
1,2134.0,50.000000,1,NaN,NaN
2,2582.0,104.000000,1,NaN,NaN
3,3000.0,150.504307,3367,918.738956,3407.204651
4,3001.0,132.500000,2,NaN,NaN


In [22]:
airbnb_price_by_zipcode.to_sql('Metrica', con=engine, if_exists="replace")

2022-05-18 21:24:58,333 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Metrica")
2022-05-18 21:24:58,335 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-18 21:24:58,339 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Metrica")
2022-05-18 21:24:58,343 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-18 21:24:58,348 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-18 21:24:58,352 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Metrica" (
	"index" BIGINT, 
	zipcode FLOAT, 
	airbnb_price_mean FLOAT, 
	airbnb_record_count BIGINT, 
	airbnb_weekly_price_mean FLOAT, 
	airbnb_monthly_price_mean FLOAT
)


2022-05-18 21:24:58,353 INFO sqlalchemy.engine.Engine [no key 0.00137s] ()
2022-05-18 21:24:58,362 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Metrica_index" ON "Metrica" ("index")
2022-05-18 21:24:58,363 INFO sqlalchemy.engine.Engine [no key 0.00092s] ()
2022-05-18 21:24:58,372 INFO sqlalchemy.engine.Engine COMMIT
2022-05-18 21:24:58,376 INFO sqlalchemy.engine.Engine BE

Generamos y ejecutamos la sentencia de SQL para unir mediante LEFT JOIN las tablas "Melbo" y "Métricas" (generada a partir del Dataset de Airbnb.

Primeramente ejecutamos un conteo de registros del Leftjoin (Desde tabla Melbo) para comparar y controlar la cantidad de registros de la tabla resultante:

In [31]:
query3 = "SELECT COUNT(*) FROM Melbo LEFT JOIN Metrica ON Metrica.zipcode=Melbo.Postcode"

In [32]:
with engine.connect() as con:
      rs = con.execute(query3)
      print(query3)
      for row in rs:
          print(row)

2022-05-18 21:28:06,620 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM Melbo LEFT JOIN Metrica ON Metrica.zipcode=Melbo.Postcode
2022-05-18 21:28:06,622 INFO sqlalchemy.engine.Engine [raw sql] ()
SELECT COUNT(*) FROM Melbo LEFT JOIN Metrica ON Metrica.zipcode=Melbo.Postcode
(13580,)


In [25]:
len(melb_df)

13580

In [ ]:
query4 = "SELECT * FROM Melbo LEFT JOIN Metrica ON Metrica.zipcode=Melbo.Postcode"

In [ ]:
# with engine.connect() as con:
#     rs = con.execute(query4)
#     print(query4)
#     for row in rs:
#        print(row)

## Ejercicio 2: 

1. Seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad. Justificar las columnas seleccionadas y las que no lo fueron.
 - Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.

2. Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico. 
  1. Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana o el mínimo.
  1. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.
  2. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.

In [44]:
melb_df.head(3)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


1.   Suburb
2.   Rooms
3.   Landsize
4.   BuildingArea
5.   YearBuilt
6.   Distance
7.   Bedroom
8.   Bathroom
9.   Car

In [41]:
import seaborn as sns

In [43]:
#g = sns.PairGrid(melb_df)

**1. Selección del subconjunto de variables relevantes:**

In [ ]:
melb_df.dtypes

Suburb            object
Address           object
Rooms              int64
Type              object
Price            float64
Method            object
SellerG           object
Date              object
Distance         float64
Postcode         float64
Bedroom2         float64
Bathroom         float64
Car              float64
Landsize         float64
BuildingArea     float64
YearBuilt        float64
CouncilArea       object
Lattitude        float64
Longtitude       float64
Regionname        object
Propertycount    float64
dtype: object

In [ ]:
pandas.options.display.float_format = '{:.2f}'.format

In [ ]:
pandas.value_counts(melb_df['Suburb'])

Reservoir         359
Richmond          260
Bentleigh East    249
Preston           239
Brunswick         222
                 ... 
Sandhurst           1
Bullengarook        1
Croydon South       1
Montrose            1
Monbulk             1
Name: Suburb, Length: 314, dtype: int64

In [ ]:
print("Cantidad de registros con datos para Suburb:", melb_df['Suburb'].value_counts().sum())

Cantidad de registros con datos para Suburb: 13580


In [ ]:
pandas.value_counts(melb_df['Regionname'])

Southern Metropolitan         4695
Northern Metropolitan         3890
Western Metropolitan          2948
Eastern Metropolitan          1471
South-Eastern Metropolitan     450
Eastern Victoria                53
Northern Victoria               41
Western Victoria                32
Name: Regionname, dtype: int64

In [ ]:
print("Cantidad de registros con datos para Regionname:", melb_df['Regionname'].value_counts().sum())

Cantidad de registros con datos para Regionname: 13580


In [ ]:
pandas.value_counts(melb_df['Type'])

h    9449
u    3017
t    1114
Name: Type, dtype: int64

In [ ]:
print("Cantidad de registros con datos para Type:", melb_df['Type'].value_counts().sum())

Cantidad de registros con datos para Type: 13580


In [ ]:
melb_df['YearBuilt'].describe()

count   8205.00
mean    1964.68
std       37.27
min     1196.00
25%     1940.00
50%     1970.00
75%     1999.00
max     2018.00
Name: YearBuilt, dtype: float64

In [ ]:
melb_df['Price'].describe()

count     13580.00
mean    1075684.08
std      639310.72
min       85000.00
25%      650000.00
50%      903000.00
75%     1330000.00
max     9000000.00
Name: Price, dtype: float64

In [ ]:
melb_df['Distance'].describe()

count   13580.00
mean       10.14
std         5.87
min         0.00
25%         6.10
50%         9.20
75%        13.00
max        48.10
Name: Distance, dtype: float64

In [ ]:
melb_df['Rooms'].describe()

count   13580.00
mean        2.94
std         0.96
min         1.00
25%         2.00
50%         3.00
75%         3.00
max        10.00
Name: Rooms, dtype: float64

A las variables categóricas realizamos un conteo de registros en el df y a las variables númericas calcularle medidas de estadistica descriptiva para analizarlas.

**2. Se agrega información del conjunto de datos de Airbnb:**

In [ ]:
merged_sales_df = melb_df.merge(
    airbnb_price_by_zipcode, how='left',
    left_on='Postcode', right_on='zipcode'
)
merged_sales_df.sample(2)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,zipcode,airbnb_price_mean,airbnb_record_count,airbnb_weekly_price_mean,airbnb_monthly_price_mean
6250,Thornbury,2/305 Rossmoyne St,3,u,510000.0,VB,Jellis,3/09/2016,6.5,3071.0,...,Darebin,-37.75910,145.02110,Northern Metropolitan,8870.0,3071.0,125.008772,114.0,675.8,2580.1
10944,Bundoora,23 Wisteria Dr,4,h,1253000.0,S,Ray,12/08/2017,12.1,3083.0,...,Whittlesea,-37.67935,145.05728,Northern Metropolitan,10175.0,3083.0,84.137931,87.0,702.5,2291.5


## Ejercicio 3:

Crear y guardar un nuevo conjunto de datos con todas las transformaciones realizadas anteriormente.

In [ ]:
#merged_sales_df.to_csv("melb_data_extended.csv", index=None)

In [ ]:
#from google.colab import files
#files.download('melb_data_extended.csv') 

## Ejercicios opcionales:

1. Armar un script en python (archivo .py) [ETL](https://towardsdatascience.com/what-to-log-from-python-etl-pipelines-9e0cfe29950e) que corra los pasos de extraccion, transformacion y carga, armando una funcion para cada etapa del proceso y luego un main que corra todos los pasos requeridos.

2. Armar un DAG en Apache Airflow que corra el ETL. (https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html)